In [ ]:
import harp
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
from functools import reduce
import cv2
import numpy as np
import utils
import aeon.io.video as video
import aeon.analysis.movies as frame_helpers
from dotmap import DotMap

harp_reader = harp.create_reader('h1-device.yml', epoch=harp.REFERENCE_EPOCH)
session_data_reader = utils.SessionData()

h1_datafolder = r'C:/temp/2024-05-07T15-22-53/HarpDataH1/'
h2_datafolder = r'C:/temp/2024-05-07T15-22-53/HarpDataH2/'
settings_datafolder = r'C:/temp/2024-05-07T15-22-53/SessionSettings/'